In [1]:
from pathlib import Path
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
from collections import defaultdict
cwd = Path(os.getcwd())
WORKING_DIR = cwd.parent
DATASET_DIR = WORKING_DIR.parent / "dataset"


In [2]:
res = cwd / (f"transcriptome/predictions_hard_mining.parquet")
preds = pd.read_parquet(res)
preds["max_type"] = preds[["prob0","prob1", "prob2", "prob3", "prob4"]].idxmax(axis=1)

counts_all = np.unique(preds["max_type"], return_counts=True)
fractions = counts_all[1] / preds.shape[0] * 100
for class_, count in zip(counts_all[0], fractions):
    print(f"Class {class_} accounts for {count:.2f}% of the predictions")

Class prob0 accounts for 99.66% of the predictions
Class prob1 accounts for 0.08% of the predictions
Class prob2 accounts for 0.16% of the predictions
Class prob3 accounts for 0.02% of the predictions
Class prob4 accounts for 0.09% of the predictions


In [3]:
tx_file = cwd / "transcriptome/gencode_customized.tsv"
tx = pd.read_csv(tx_file, sep="\t")
tx["ids"] = tx.index
mapping_to_genes = tx["gene_id"].to_dict()

In [4]:
#we generate or load the unusable positions  in the transcriptome, which correspond to sequences long 51 already present in the training dataset
unusable_path = cwd / "transcriptome/unusable_sequences.pickle"

if unusable_path.exists():
    print(f"Loading existing unusable sequences from {unusable_path}")
    with open(unusable_path, "rb") as f:
        unusable_sequences = pickle.load(f)
else:
    print(f"File not found. Generating unusable sequences...")
    
    # Load the dataset and process sequences
    path_full_dataset = DATASET_DIR / "full_dataset_final_training.pickle"
    with open(path_full_dataset, "rb") as f:
        full_dataset = pickle.load(f)
    
    resized_sequences = []
    for key, value in full_dataset.items():
        for i in range(len(value)):
            seq = value[i]
            seq = seq[50:151-50]
            resized_sequences.append(seq)

    resized_sequences = set(resized_sequences)

    # Find unusable sequences
    unusable_sequences = defaultdict(list)
    for index, row in tx.iterrows():
        seq = row["sequence"]
        c_pos = np.where(np.array([x for x in seq]) == "C")[0]
        for pos in c_pos:
            cut_seq = seq[pos-25:pos+26]
            if cut_seq in resized_sequences:
                unusable_sequences[index].append(int(pos))
    unusable_sequences = dict(unusable_sequences)

    with open(unusable_path, "wb") as f:
        pickle.dump(unusable_sequences, f)
    
    print(f"Unusable sequences generated and saved to {unusable_path}")

unrolled_unusable_sequences = set()
for key, value in unusable_sequences.items():
    for pos in value:
        unrolled_unusable_sequences.add((key, pos))

# Add an 'is_unusable' column that's True when the pair exists in the set
# Create a list of tuples from the DataFrame columns
tx_pos_pairs = list(zip(preds['tx_idx'], preds['center_pos']))

# Check if each pair is in the set (faster than apply for large DataFrames)
preds['is_unusable'] = [pair in unrolled_unusable_sequences for pair in tx_pos_pairs]
preds = preds.loc[preds["max_type"] != "prob0"]
preds["prob_max"] = preds[["prob1", "prob2", "prob3", "prob4"]].max(axis=1)


Loading existing unusable sequences from /home/saitto/rna_modifications_project/experiments/transcriptome_analysis/transcriptome/unusable_sequences.pickle


In [5]:
#to save final
nice_names   = dict(prob0="Negatives", prob1="I", prob2="II",
                    prob3="III", prob4="IV")

preds_to_save = preds.copy()
preds_to_save.drop(columns=["prob0", "prob1", "prob2", "prob3", "prob4"], inplace=True)
tx_to_keep = tx[tx["ids"].isin(preds_to_save["tx_idx"].tolist())]
tx_to_keep = tx_to_keep.merge(preds_to_save, left_on="ids", right_on="tx_idx", how="inner")
tx_to_keep.drop(columns=["ids", "tx_idx"], inplace=True)
tx_to_keep["sequence"] = tx_to_keep["sequence"].apply(lambda x: x[25:])
tx_to_keep["center_pos"] = tx_to_keep["center_pos"].apply(lambda x: x - 25)
tx_to_keep["max_type"] = tx_to_keep["max_type"].map(nice_names)
tx_to_keep.rename(columns={"max_type": "Type", "prob_max": "probability", "is_unusable": "in_train_or_test_sets", "center_pos": "position"}, inplace=True)

In [ ]:
tx_to_keep_no_seq = tx_to_keep.copy()
tx_to_keep_no_seq.drop(columns=["sequence"], inplace=True)

tx_to_keep_no_seq.to_csv(cwd / "analysis_results/m5C_predictions.tsv.gz",
               sep="\t", index=False, compression="gzip")

tx_to_keep_no_seq.to_csv(cwd / "analysis_results/m5C_predictions.tsv",
               sep="\t", index=False)



In [10]:

tx_to_keep_no_seq.to_excel(cwd / "analysis_results/m5C_predictions.xlsx", index=False)

# #save to keep with sequence in normal tsv format
# tx_to_keep.to_csv(cwd / "analysis_results/m5C_predictions_with_seq.tsv",
#                sep="\t", index=False, compression=None)